In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

In [4]:
df=pd.read_csv('./data/processed.cleveland.data.csv')

In [5]:
df.head()

,63.0,1.0,1.0.1,145.0,233.0,1.0.2,2.0,150.0,0.0,2.3,3.0,0.0.1,6.0,0
0,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
1,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
2,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
3,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0
4,56.0,1.0,2.0,120.0,236.0,0.0,0.0,178.0,0.0,0.8,1.0,0.0,3.0,0


As our data don't have column names let's assign them names. The names are from where the dataset is taken from  

In [6]:
df.columns = [
'age',
'sex',
'chest_pain',
'blood pressure',
'serum_cholestoral',
'fasting_blood_sugar',
'electrocardiographic',
'max_heart_rate',
'induced_angina',
'ST_depression',
'slope',
'vessels',
'thal',
'diagnosis']

df.head()

,age,sex,chest_pain,blood pressure,serum_cholestoral,fasting_blood_sugar,electrocardiographic,max_heart_rate,induced_angina,ST_depression,slope,vessels,thal,diagnosis
0,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
1,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
2,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
3,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0
4,56.0,1.0,2.0,120.0,236.0,0.0,0.0,178.0,0.0,0.8,1.0,0.0,3.0,0


## Dealing with the missing data

In [9]:
# dtypes tells datatype of each column
df.dtypes

age                     float64
sex                     float64
chest_pain              float64
blood pressure          float64
serum_cholestoral       float64
fasting_blood_sugar     float64
electrocardiographic    float64
max_heart_rate          float64
induced_angina          float64
ST_depression           float64
slope                   float64
vessels                  object
thal                     object
diagnosis                 int64
dtype: object

We can see both vessels and thal have 'object' datatype so they might have both numbers and some other characters. Let's print all see what is going

In [10]:
df['vessels'].unique()

array(['3.0', '2.0', '0.0', '1.0', '?'], dtype=object)

In [12]:
df['thal'].unique()

array(['3.0', '7.0', '6.0', '?'], dtype=object)

In [17]:
# loc helps to access rows and columns by labels or boolean
len(df.loc[(df['vessels']=='?')
    | (df['thal']=='?')])

6